# compute change of fundamentals

In [1]:
import wrds
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import numpy as np

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

pd.set_option('display.max_columns', 100) 
pd.set_option('display.max_colwidth', 100) 
pd.set_option('display.width', 1000)  

conn = wrds.Connection(wrds_username='george_gkolemis')

#Compustat
query = """
SELECT conm, cusip, tic, tpci, datadate, exchg, prccd, ajexdi

FROM  comp_na_daily_all.secd

WHERE 
    datadate BETWEEN '1998-12-20' AND '2011-01-01' -- '2010-12-20' AND '2016-01-01'
    AND tpci IN ('0') -- Limiting the search to stocks only
    AND exchg IN (11) -- Limiting the search to NYSE (11) or/and NASDAQ (14)
    AND iid = '01' --Some companies have multiple classes of stocks. Here taking only class A
"""

df_price = conn.raw_sql(query, params=None)

df_price['cusip'] = df_price['cusip'].str[:-1]

df_price['adjusted_price'] = df_price['prccd'] / df_price['ajexdi']
df_price['datadate'] = pd.to_datetime(df_price['datadate'])
df_price

Loading library list...
Done


,conm,cusip,tic,tpci,datadate,exchg,prccd,ajexdi,adjusted_price
0,AAR CORP,00036110,AIR,0,1998-12-21,11,23.2500,1.00,23.2500
1,AAR CORP,00036110,AIR,0,1998-12-22,11,24.1875,1.00,24.1875
2,AAR CORP,00036110,AIR,0,1998-12-23,11,24.1250,1.00,24.1250
3,AAR CORP,00036110,AIR,0,1998-12-24,11,23.6250,1.00,23.6250
4,AAR CORP,00036110,AIR,0,1998-12-28,11,23.1250,1.00,23.1250
...,...,...,...,...,...,...,...,...,...
340710,KITE REALTY GROUP TRUST,49803T30,KRG,0,2010-12-27,11,5.6100,0.25,22.4400
340711,KITE REALTY GROUP TRUST,49803T30,KRG,0,2010-12-28,11,5.5800,0.25,22.3200
340712,KITE REALTY GROUP TRUST,49803T30,KRG,0,2010-12-29,11,5.5500,0.25,22.2000
340713,KITE REALTY GROUP TRUST,49803T30,KRG,0,2010-12-30,11,5.5000,0.25,22.0000


In [2]:
adj_price_df = df_price[['cusip', 'datadate', 'adjusted_price']].copy().set_index(['datadate', 'cusip']).unstack()
#adj_price_df = adj_price_df.ffill()

return_df = adj_price_df/adj_price_df.shift(1)-1
return_df.columns = return_df.columns.get_level_values(1)
#return_df

In [3]:
monthly_return = (return_df+1).rolling(21).apply(np.prod, raw=True) - 1
monthly_volatility = return_df.rolling(21).std()
monthly_sharpe = monthly_return/monthly_volatility
monthly_return

cusip,00036110,00095710,00105510,00108410,00120410,00125010,00130H10,00154710,00163T10,00169310,00173510,00174410,00184X10,00191G10,00191U10,00195750,00206R10,00209A10,00244410,00245110,00247410,00253G10,00282410,00289620,00299Y94,00392410,00423910,00434L10,00452710,00483310,00489Q10,00493310,00508Y10,00511310,00737L10,00739W10,00751Y10,00756610,00758510,00763M10,00766T10,00776710,00780010,00786910,00817Y10,00819010,00825210,00845V10,00846U10,00847410,...,G8761510,G8766E10,G8915Z10,G8994E10,G9075110,G9108L17,G9144210,G9144B10,G9319H10,G9618E10,G9782210,G9829410,H0130110,H1135610,H1467J10,H2906T10,H8817H10,H8999Z93,L3466T10,M4016810,M8737E10,N0098510,N1251N10,N2093520,N2925S10,N5374510,N7248214,N8240510,P1699413,P3107610,P6065Y10,P6465810,P7482310,V7780T10,V8726M10,Y0436Q10,Y0553W10,Y1771G10,Y1820X10,Y1968P12,Y2065G12,Y2066G10,Y6215914,Y6219711,Y6226740,Y6476W10,Y7388L10,Y7542C13,Y8564M10,Y8565J10
datadate,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1998-12-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-12-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-12-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-12-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-12-27,0.137423,0.130545,0.055391,0.084837,-0.025154,NaN,0.116514,0.239065,0.060484,NaN,NaN,0.088235,-0.036458,0.002721,0.235294,NaN,0.039446,NaN,0.052345,NaN,0.050324,NaN,0.008944,0.185931,NaN,NaN,-0.004889,-0.061862,NaN,NaN,0.143939,NaN,0.101745,NaN,0.086383,0.128514,-8.656716e-03,NaN,NaN,NaN,0.066768,0.083333,0.027505,NaN,0.013434,NaN,0.110519,NaN,0.153550,-0.059187,...,NaN,0.004187,NaN,0.146365,NaN,0.091496,0.102165,NaN,0.026531,0.042869,0.036286,0.122914,0.015904,0.030819,0.039879,0.

# Functions for calculating metrics

In [4]:
zscore = lambda x: (x - x.mean())/x.std()

In [5]:
#DataFrame to Store Features and Target
#add target
target= monthly_return.shift(-21).stack(dropna=False).copy().to_frame()
target.columns = ['monthly_return']
target=target.groupby(level=0).transform(zscore)
target

monthly_return
datadate   cusip                   
1998-12-21 00036110       -1.069562
           00095710       -0.622815
           00105510       -0.155069
           00108410        0.892997
           00120410       -0.418490
...                             ...
2010-12-31 Y6476W10             NaN
           Y7388L10             NaN
           Y7542C13             NaN
           Y8564M10             NaN
           Y8565J10             NaN

[7662798 rows x 1 columns]

In [6]:
def RSI(return_df, n):
    '''
    data frame of returns (col: stock, row: dates)
    n: number of days to look back 

    returns PIT RSI observed in the previous n_days
    
    '''
    #Keep magnitude of gain (up) or losses (down) for calculating avg
    up=return_df.map(lambda x : x if x>0 else 0, na_action= 'ignore')
    down=return_df.map(lambda x : -x if x<0 else 0, na_action= 'ignore')

    avg_up=up.shift(1).rolling(n).mean()
    avg_down=down.shift(1).rolling(n).mean()
    
    n_day_rsi = 100 - (100/ (1+ avg_up/avg_down))
    return n_day_rsi

In [7]:
def TSI(price, smooth_1 = 25, smooth_2 = 13):
    '''
    Double Smoothed PC
    ------------------
    PC = Current Price minus Prior Price
    First Smoothing = 25-period EMA of PC
    Second Smoothing = 13-period EMA of 25-period EMA of PC
    
    Double Smoothed Absolute PC
    ---------------------------
    Absolute Price Change |PC| = Absolute Value of Current Price minus Prior Price
    First Smoothing = 25-period EMA of |PC|
    Second Smoothing = 13-period EMA of 25-period EMA of |PC|
    
    TSI = 100 x (Double Smoothed PC / Double Smoothed Absolute PC)
    data frame of price (col: stock, row: dates)
    smooth_1:
    smooth_2: 

    returns PIT RSI observed in the previous n_days
    '''
    pc= price-price.shift(1)
    apc= abs(pc)
    #double smooth pc
    ema_pc = pc.ewm(span=smooth_1, axis=0, adjust=False).mean()
    ema_ema_pc = ema_pc.ewm(span=smooth_2, axis=0, adjust=False).mean()

    #double smooth apc
    ema_apc = apc.ewm(span=smooth_1, axis=0, adjust=False).mean()
    ema_ema_apc = ema_apc.ewm(span=smooth_2, axis=0, adjust=False).mean()
    #calculate tsi
    tsi = ema_ema_pc/ema_ema_apc * 100

    pit_tsi= tsi.shift(1)

    #only return days with sufficient smoothing
    return pit_tsi.iloc[smooth_1:, :]

In [8]:
def rolling_momentum(dataframe, months=12, exclude_last_month=True, 
                     smoothing_type='sma', smoothing_window=3, 
                     weight_recent=True):

    """
    Calculates the rolling momentum with optional smoothing and recent-weighting adjustments.

    Args:
    - dataframe (pd.DataFrame): DataFrame with daily price data, columns as stock tickers, rows as dates.
    - months (int): Number of months to calculate momentum over (default is 12).
    - exclude_last_month (bool): Whether to exclude the most recent month in calculation (default is True).
    - smoothing_type (str): Type of smoothing to apply; 'sma' (simple moving average) or 'ewma' (exponentially weighted moving average).
    - smoothing_window (int): Window size for smoothing in SMA (default is 3).
    - weight_recent (bool): If True, applies recent weighting using EWMA with specified span.
    - ewm_span (int): The span parameter for EWMA, controlling the decay rate (default is 3).

    Returns:
    - momentum_df (pd.DataFrame): DataFrame containing the rolling momentum for each stock.
    """

    dataframe = dataframe.copy()
    dataframe.index = pd.to_datetime(dataframe.index)

    # Calculate momentum over the specified period
    if exclude_last_month:
        shifted_df = dataframe.shift(21)  # assuming 21 trading days per month
        momentum_df = (shifted_df - dataframe.shift(252)) / dataframe.shift(252)
    else:
        momentum_df = (dataframe - dataframe.shift(252)) / dataframe.shift(252)

    # Apply smoothing based on the chosen method
    if smoothing_type == 'sma':
        smoothed_momentum = momentum_df.rolling(window=smoothing_window, min_periods=1).mean()
    elif smoothing_type == 'ewma':
        smoothed_momentum = momentum_df.ewm(span=smoothing_window, adjust=False).mean()
    else:
        smoothed_momentum = momentum_df

    return smoothed_momentum

In [9]:
def generate_momentum_features(dataframe, sma_windows=[3, 5, 10, 50, 100], ema_windows=[3, 5, 10, 50, 100], months=12):
    """
    Generates momentum features for specified SMA and EMA windows.
    
    Args:
    - dataframe (pd.DataFrame): DataFrame with daily price data, columns as stock tickers, rows as dates.
    - sma_windows (list): List of windows for SMA-based momentum features.
    - ema_windows (list): List of windows for EMA-based momentum features.
    - months (int): Number of months to calculate momentum over (default is 12).

    Returns:
    - features_df (pd.DataFrame): DataFrame with momentum features and monthly returns.
    """
    # Copy and setup
    adj_price_df = dataframe.copy()
    adj_price_df.index = pd.to_datetime(adj_price_df.index)
    momentum_features = {}

    # Generate SMA features
    for sma_window in sma_windows:
        col_name = f"Momentum_SMA_{sma_window}"
        # Stack to align (date, ticker) structure
        momentum_features[col_name] = rolling_momentum(adj_price_df, smoothing_type='sma', smoothing_window=sma_window).stack()
        #print(f"{col_name} shape:", momentum_features[col_name].shape)

    # Generate EMA features
    for ema_window in ema_windows:
        col_name = f"Momentum_EMA_{ema_window}"
        # Stack to align (date, ticker) structure
        momentum_features[col_name] = rolling_momentum(adj_price_df, smoothing_type='ewma', smoothing_window=ema_window).stack()
        #print(f"{col_name} shape:", momentum_features[col_name].shape)

    # Combine features into a single DataFrame (columns become feature names, index is date and ticker)
    features_df = pd.concat(momentum_features, axis=1)
    features_df.columns = list(momentum_features.keys())
    
    #PIT
    features_df = features_df.shift(1)
    return features_df

In [10]:
rsi = RSI(return_df, 15)
rsi = rsi.transform(zscore , axis=1)
rsi = rsi.stack().to_frame()
rsi.columns = ['rsi']

In [11]:
tsi = TSI(return_df, 15)
tsi = tsi.transform(zscore , axis=1)
tsi = tsi.stack().to_frame()
tsi.columns = ['tsi']

In [12]:
momentum = generate_momentum_features(adj_price_df, sma_windows=[], ema_windows=[3, 10])
#get zscore
momentum = momentum.groupby('cusip').transform(lambda x: x.pct_change())
momentum = momentum.groupby(level=0).transform(zscore)

/Users/georgiosgkolemis/Library/Python/3.9/lib/python/site-packages/numpy/core/_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/Users/georgiosgkolemis/Library/Python/3.9/lib/python/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


In [13]:
price_features = pd.concat([target, tsi, rsi, momentum], axis=1).dropna()
price_features = price_features.clip(-3,3)
price_features

monthly_return       tsi       rsi  Momentum_EMA_3  Momentum_EMA_10
datadate   cusip                                                                        
1999-12-27 00036110        0.348065 -0.214940 -0.210398       -0.000285         0.032183
           00095710       -0.136851 -0.401006  0.132732       -0.006469         0.030549
           00105510       -0.449021 -0.188962 -0.870259       -0.002833         0.030758
           00108410       -0.620943  0.056545  0.310023       -0.035860        -0.004531
           00120410       -0.086330 -0.233944 -1.200586        0.016202         0.040853
...                             ...       ...       ...             ...              ...
2010-12-01 Y6219711       -0.565342 -0.025970 -2.008413        0.022683        -0.015217
           Y6226740       -0.255423  0.040511 -0.678095       -0.042151        -0.177435
           Y7388L10        0.672787  0.069709 -1.314472        0.023158        -0.015010
           Y8564M10       -0.347285  0.199572  1.050337        0.164240         0.048174
           Y8565J10       -1.165042 -0.483407  0.881048        0.028550        -0.000784

[3542536 rows x 5 columns]

### Fundamental Factors

Identifiers for GIC sectors

Energy 10 <br>
Materials 15<br>
Industrials 20<br>
Consumer Discretionary 25<br>
Consumer Staples 30<br>
Health Care 35<br>
Financials 40<br>
Information Technology 45<br>
Telecommunication 50<br>
Utilities 55<br>
Real Estate 60

In [14]:
conn = wrds.Connection(wrds_username='george_gkolemis')

#Compustat
query = """
SELECT * --cusip, public_date, bm, 1/pe_inc as ep, 1/ps AS sp, gpm, npm, cfm, de_ratio, intcov_ratio, inv_turn, 1/pcf as cfp, fcf_ocf

FROM wrdsapps_finratio.firm_ratio

WHERE 
    public_date BETWEEN '1998-12-20' AND '2011-01-01' -- '2010-12-20' AND '2016-01-01'
    --AND gsector IN ('35') -- Adjust sector according the numbers above
    
"""

df_factors = conn.raw_sql(query, params=None)

#df_factors.columns = ['cusip', 'datadate', 'B/M', 'E/P', 'S/P','GPM', 'NPM', 'CFM', 'D/E', 'ICR', 'Inv_Turn', 'CF/P', 'FCF/OCF']
df_factors['datadate'] = pd.to_datetime(df_factors['public_date'])

df_factors

Loading library list...
Done


,gvkey,permno,adate,qdate,public_date,capei,be,bm,evm,pe_op_basic,pe_op_dil,pe_exi,pe_inc,ps,pcf,dpr,npm,opmbd,opmad,gpm,ptpm,cfm,roa,roe,roce,efftax,aftret_eq,aftret_invcapx,aftret_equity,pretret_noa,pretret_earnat,gprof,equity_invcap,debt_invcap,totdebt_invcap,capital_ratio,int_debt,int_totdebt,cash_lt,invt_act,rect_act,debt_at,debt_ebitda,short_debt,curr_debt,lt_debt,profit_lct,ocf_lct,cash_debt,fcf_ocf,lt_ppent,dltt_be,debt_assets,debt_capital,de_ratio,intcov,intcov_ratio,cash_ratio,quick_ratio,curr_ratio,cash_conversion,inv_turn,at_turn,rect_turn,pay_turn,sale_invcap,sale_equity,sale_nwc,rd_sale,adv_sale,staff_sale,accrual,ret_crsp,gsector,gicdesc,mktcap,price,ptb,peg_trailing,divyield,ffi5_desc,ffi5,ffi10_desc,ffi10,ffi12_desc,ffi12,ffi17_desc,ffi17,ffi30_desc,ffi30,ffi38_desc,ffi38,ffi48_desc,ffi48,ffi49_desc,ffi49,ticker,cusip,datadate
0,001004,54594,1998-05-31,1998-08-31,1998-12-31,37.600572,346.620,0.565106,10.560636,17.342195,NaN,17.728960,17.728960,0.800235,18.166231,0.244166,0.045907,0.102657,0.084463,0.208647,0.065903,0.064100,0.137903,0.117503,0.156623,0.302989,0.131831,0.114666,0.131831,0.177375,0.131867,0.262698,0.645473,0.354527,0.376757,0.354527,0.098287,0.092487,0.055443,0.563705,0.333936,0.263290,2.037051,0.059002,0.429172,0.451348,0.548642,0.235426,0.061732,0.410430,4.449729,0.489514,0.548923,0.491587,1.216916,3.373570,4.367131,0.129185,1.344787,3.082286,156.920749,2.434150,1.259054,5.192372,6.170170,1.801654,2.791217,2.566618,0.000000,0.000000,0.0,0.002421,-0.054455,None,None,661.886625,23.8750,1.909545,0.436787,0.014241,CNSMR,1.0,SHOPS,7.0,SHOPS,9.0,MACHN,11.0,WHLSL,26.0,WHLSL,33.0,WHLSL,41.0,WHLSL,42.0,AIR,00036110,1998-12-31
1,001004,54594,1998-05-31,1998-11-30,1999-01-31,27.273218,355.274,0.510643,10.110427,13.644366,NaN,13.839286,13.839286,0.553497,11.512687,0.237662,0.041082,0.092664,0.076621,0.188501,0.059029,0.057125,0.135947,0.119083,0.157356,0.302989,0.133616,0.115386,0.133616,0.183105,0.132326,0.261588,0.631031,0.368969,0.378146,0.368969,0.098677,0.096282,0.064098,0.559088,0.333981,0.262472,2.041111,0.024269,0.436131,0.455696,0.524641,0.272201,0.061732,0.453190,4.585473,0.520529,0.562001,0.506844,1.283112,3.255940,4.207510,0.146970,1.307375,2.965161,134.710422,2.771474,1.387730,5.717165,6.408382,1.999323,3.168344,2.881065,0.000000,0.000000,0.0,-0.010017,-0.184921,None,None,533.451875,19.3750,1.501522,0.380628,0.017548,CNSMR,1.0,SHOPS,7.0,SHOPS,9.0,MACHN,11.0,WHLSL,26.0,WHLSL,33.0,WHLSL,41.0,WHLSL,42.0,AIR,00036110,1999-01-31
2,001004,54594,1998-05-31,1998-11-30,1999-02-28,21.290706,355.274,0.510643,10.110427,10.651408,NaN,10.803571,10.803571,0.432085,8.987324,0.237662,0.041082,0.092664,0.076621,0.188501,0.059029,0.057125,0.135947,0.119083,0.157356,0.302989,0.133616,0.115386,0.133616,0.183105,0.132326,0.261588,0.631031,0.368969,0.378146,0.368969,0.098677,0.096282,0.064098,0.559088,0.333981,0.262472,2.041111,0.024269,0.436131,0.455696,0.524641,0.272201,0.061732,0.453190,4.585473,0.520529,0.562001,0.506844,1.283112,3.255940,4.207510,0.146970,1.307375,2.965161,134.710422,2.771474,1.387730,5.717165,6.408382,1.999323,3.168344,2.881065,0.000000,0.000000,0.0,-0.010017,-0.219355,None,None,416.436625,15.1250,1.172156,0.297135,0.022479,CNSMR,1.0,SHOPS,7.0,SHOPS,9.0,MACHN,11.0,WHLSL,26.0,WHLSL,33.0,WHLSL,41.0,WHLSL,42.0,AIR,00036110,1999-02-28
3,001004,54594,1998-05-31,1998-11-30,1999-03-31,25.073765,355.274,0.510643,10.110427,12.544014,NaN,12.723214,12.723214,0.508860,10.584245,0.237662,0.041082,0.092664,0.076621,0.188501,0.059029,0.057125,0.135947,0.119083,0.157356,0.302989,0.133616,0.115386,0.133616,0.183105,0.132326,0.261588,0.631031,0.368969,0.378146,0.368969,0.098677,0.096282,0.064098,0.559088,0.333981,0.262472,2.041111,0.024269,0.436131,0.455696,0.524641,0.272201,0.061732,0.453190,4.585473,0.520529,0.562001,0.506844,1.283112,3.255940,4.207510,0.146970,1.307375,2.965161,134.710422,2.771474,1.387730,5.717165,6.408382,1.999323,3.168344,2.881065,0.000000,0.000000,0.0,-0.010017,0.177686,None,None,490.43

In [15]:
df_factors = df_factors.drop(columns=['gvkey', 'permno', 'adate', 'qdate', 'public_date', 'gsector', 'gicdesc', 'ffi10_desc',"ffi5_desc", "ffi5", "ffi10",
    "ffi12_desc", "ffi12",
    "ffi17_desc", "ffi17",
    "ffi30_desc", "ffi30",
    "ffi38_desc", "ffi38",
    "ffi48_desc", "ffi48",
    "ffi49_desc", "ffi49",
    "ticker"])

In [16]:
#for some reason there are duplicates and I take the average if thats the case (5% of the dataset)
df_factors = df_factors.groupby(['datadate', 'cusip'], as_index=False).mean()

# Calculate change in factors
factor_columns = df_factors.columns.difference(['datadate', 'cusip'])
#df_factors[factor_columns] = df_factors.groupby('cusip')[factor_columns].transform(lambda x: x.pct_change())

#calculate zscores
fundamental_f = df_factors.copy().set_index(['datadate', 'cusip'])
fundamental_f = fundamental_f.groupby(level=0).transform(zscore)

#uncomment and list the factors you want to omit
#fundamental_f = fundamental_f.drop(columns=['E/P', 'D/E', 'B/M', 'CFM', 'ICR', 'GPM'])

fundamental_f

capei        be        bm       evm  pe_op_basic  pe_op_dil    pe_exi    pe_inc        ps       pcf       dpr       npm     opmbd     opmad       gpm      ptpm       cfm       roa       roe      roce    efftax  aftret_eq  aftret_invcapx  aftret_equity  pretret_noa  pretret_earnat     gprof  equity_invcap  debt_invcap  totdebt_invcap  capital_ratio  int_debt  int_totdebt   cash_lt  invt_act  rect_act   debt_at  debt_ebitda  short_debt  curr_debt   lt_debt  profit_lct   ocf_lct  cash_debt   fcf_ocf  lt_ppent   dltt_be  debt_assets  debt_capital  de_ratio    intcov  intcov_ratio  cash_ratio  quick_ratio  curr_ratio  cash_conversion  inv_turn   at_turn  rect_turn  pay_turn  sale_invcap  sale_equity  sale_nwc   rd_sale  adv_sale  staff_sale   accrual  ret_crsp    mktcap     price       ptb  peg_trailing  divyield
datadate   cusip                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
1998-12-31 00036020 -0.009390 -0.234652 -0.405436 -0.000764    -0.075640        NaN -0.116016 -0.094274 -0.256796  0.013180 -0.074207  0.040473  0.037561  0.038193  0.026733  0.040576  0.039304  0.716899 -0.001634  0.061290 -0.073625   0.060746        0.032657       0.066336     0.018145        0.463255  0.366192      -0.074853     0.126728       -0.043825       0.097157 -0.047775    -0.067521 -0.219415  0.678379  0.858483  0.220750    -0.012738   -0.893601  -0.307497  0.761233    0.301236  0.327605   0.194997  0.015737 -0.013887 -0.020718     0.049625      0.059925 -0.057493 -0.001506     -0.030150   -0.314098    -0.243239   -0.191211        -0.076118 -0.084977  1.149046  -0.075533 -0.022826     0.075516     0.100201 -0.011501 -0.036468 -0.131736   -0.310811 -0.021211 -0.090145 -0.158251 -0.022411 -0.140990     -0.315398       NaN
           00036110  0.018522 -0.082964 -0.261739  0.006041     0.128021        NaN  0.131007  0.143351 -0.238932  0.214962 -0.008868  0.040436  0.037543  0.038381  0.026834  0.040528  0.039063  0.320601 -0.004098  0.038688 -0.046153   0.037907        0.026919       0.042460     0.014018        0.333302 -0.151360      -0.041281     0.090873       -0.051147       0.075127 -0.046503    -0.052620 -0.207444  1.409629 -0.121073  0.152710     0.001707   -0.811225  -0.601510  0.631862    0.182441  0.144871   0.092661  0.015737 -0.013663 -0.023863     0.026538      0.047526 -0.059488 -0.007483     -0.036306   -0.288350    -0.234370   -0.016912        -0.007180 -0.097461  0.191177  -0.081602 -0.037448    -0.021906    -0.039616 -0.030366 -0.036468 -0.131736   -0.310811  0.313975 -0.343485 -0.105677 -0.005817 -0.286277     -0.213099 -0.348241
           00040010 -0.013536 -0.212822  0.028095  0.001425     0.049632  -0.016430  0.040902  0.056675 -0.197247 -0.088027 -0.074207  0.041120  0.041117  0.041124  0.034646  0.041475  0.040431 -0.123929 -0.004805  0.040688 -0.050863   0.027791        0.016957       0.031886          NaN             NaN -0.746020       0.435224    -0.418029       -0.122710      -0.237544       NaN          NaN -0.204640       NaN       NaN -0.896138    -0.017430    1.756038        NaN -1.055518         NaN       NaN   0.044035       NaN -0.011531 -0.036291     1.235241      0.417330  0.414879       NaN           NaN         NaN          NaN         NaN     

In [17]:
price_and_fundamentals = pd.merge(price_features, fundamental_f, on =['datadate','cusip'], how='outer')
price_and_fundamentals[fundamental_f.columns] = price_and_fundamentals.groupby('cusip')[fundamental_f.columns].ffill()

price_and_fundamentals

monthly_return  tsi  rsi  Momentum_EMA_3  Momentum_EMA_10     capei        be        bm       evm  pe_op_basic  pe_op_dil    pe_exi    pe_inc        ps       pcf       dpr       npm     opmbd     opmad       gpm      ptpm       cfm       roa       roe      roce    efftax  aftret_eq  aftret_invcapx  aftret_equity  pretret_noa  pretret_earnat     gprof  equity_invcap  debt_invcap  totdebt_invcap  capital_ratio  int_debt  int_totdebt   cash_lt  invt_act  rect_act   debt_at  debt_ebitda  short_debt  curr_debt   lt_debt  profit_lct   ocf_lct  cash_debt   fcf_ocf  lt_ppent   dltt_be  debt_assets  debt_capital  de_ratio    intcov  intcov_ratio  cash_ratio  quick_ratio  curr_ratio  cash_conversion  inv_turn   at_turn  rect_turn  pay_turn  sale_invcap  sale_equity  sale_nwc   rd_sale  adv_sale  staff_sale   accrual  ret_crsp    mktcap     price       ptb  peg_trailing  divyield
datadate   cusip                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
1998-12-31 00036020             NaN  NaN  NaN             NaN              NaN -0.009390 -0.234652 -0.405436 -0.000764    -0.075640        NaN -0.116016 -0.094274 -0.256796  0.013180 -0.074207  0.040473  0.037561  0.038193  0.026733  0.040576  0.039304  0.716899 -0.001634  0.061290 -0.073625   0.060746        0.032657       0.066336     0.018145        0.463255  0.366192      -0.074853     0.126728       -0.043825       0.097157 -0.047775    -0.067521 -0.219415  0.678379  0.858483  0.220750    -0.012738   -0.893601  -0.307497  0.761233    0.301236  0.327605   0.194997  0.015737 -0.013887 -0.020718     0.049625      0.059925 -0.057493 -0.001506     -0.030150   -0.314098    -0.243239   -0.191211        -0.076118 -0.084977  1.149046  -0.075533 -0.022826     0.075516     0.100201 -0.011501 -0.036468 -0.131736   -0.310811 -0.021211 -0.090145 -0.158251 -0.022411 -0.140990     -0.315398       NaN
           00036110             NaN  NaN  NaN             NaN              NaN  0.018522 -0.082964 -0.261739  0.006041     0.128021        NaN  0.131007  0.143351 -0.238932  0.214962 -0.008868  0.040436  0.037543  0.038381  0.026834  0.040528  0.039063  0.320601 -0.004098  0.038688 -0.046153   0.037907        0.026919       0.042460     0.014018        0.333302 -0.151360      -0.041281     0.090873       -0.051147       0.075127 -0.046503    -0.052620 -0.207444  1.409629 -0.121073  0.152710     0.001707   -0.811225  -0.601510  0.631862    0.182441  0.144871   0.092661  0.015737 -0.013663 -0.023863     0.026538      0.047526 -0.059488 -0.007483     -0.036306   -0.288350    -0.234370   -0.016912        -0.007180 -0.097461  0.191177  -0.081602 -0.037448    -0.021906    -0.039616 -0.030366 -0.036468 -0.131736   -0.310811  0.313975 -0.343485 -0.105677 -0.005817 -0.286277     -0.213099 -0.348241
           00040010             NaN  NaN  NaN             NaN              NaN -0.013536 -0.212822  0.028095  0.001425     0.049632  -0.016430  0.040902  0.056675 -0.197247 -0.088027 -0.074207  0.041120  0.041117  0.041124  0.034646  0.041475  0.040431 -0.123929 -0.004805  0.040688 -0.050863   0.027791        0.016957       0.031886          NaN             NaN -0.746020       0.435224    -0.418029       -

### Analyst recommendations

STRONG BUY 1<br>
BUY 2<br>
HOLD 3<br>
UNDEPERFORM 4<br>
SELL 5

In [18]:
conn = wrds.Connection(wrds_username='george_gkolemis')

#Compustat
query = """
SELECT cusip, anndats, ireccd

FROM  tr_ibes.recddet

WHERE 
    anndats BETWEEN '1998-12-20' AND '2011-01-01' -- '2010-12-20' AND '2016-01-01' 
"""

df_rec = conn.raw_sql(query, params=None)
df_rec = df_rec.dropna()
#turn ratings into integers and rename the date column to be the same as the other frames
df_rec['ireccd'] = df_rec['ireccd'].astype(int)
df_rec['anndats'] = pd.to_datetime(df_rec['anndats'])
df_rec.columns = ['cusip', 'datadate', 'ireccd']

df_rec = df_rec.groupby(['datadate', 'cusip']).mean().reset_index()
df_rec

Loading library list...
Done


,datadate,cusip,ireccd
0,1998-12-21,00176510,2.0
1,1998-12-21,00195710,1.0
2,1998-12-21,00910410,2.0
3,1998-12-21,01642T10,3.0
4,1998-12-21,01736110,1.0
...,...,...,...
1268957,2010-12-31,FK652979,1.0
1268958,2010-12-31,FKB00LR0,1.0
1268959,2010-12-31,FSB17KMY,1.0
1268960,2010-12-31,SNB1P582,2.0


In [19]:
# compute change and zscore of recommendations
df_rec['ireccd'] = df_rec.groupby('cusip')['ireccd'].pct_change()
df_rec = df_rec.set_index(['datadate', 'cusip'])
df_rec = df_rec.groupby(level=0).transform(zscore)

target_features = pd.merge(price_and_fundamentals, df_rec, on =['datadate','cusip'], how='outer')
#forward filling assuming that recommendation will change when it should change
target_features['ireccd'] = target_features.groupby('cusip')['ireccd'].ffill()
#target_features['ireccd'] = target_features['ireccd'].fillna(0)


#start from the same date as target features
min_date = price_features.index.get_level_values(0)[0]
target_features = target_features[target_features.index.get_level_values(0) >= min_date]
target_features = target_features.dropna()
#cap zscores to -3 and 3
target_features = target_features.clip(-3,3)
target_features

/Users/georgiosgkolemis/Library/Python/3.9/lib/python/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


monthly_return       tsi       rsi  Momentum_EMA_3  Momentum_EMA_10     capei        be        bm       evm  pe_op_basic  pe_op_dil    pe_exi    pe_inc        ps       pcf       dpr       npm     opmbd     opmad       gpm      ptpm       cfm       roa       roe      roce    efftax  aftret_eq  aftret_invcapx  aftret_equity  pretret_noa  pretret_earnat     gprof  equity_invcap  debt_invcap  totdebt_invcap  capital_ratio  int_debt  int_totdebt   cash_lt  invt_act  rect_act   debt_at  debt_ebitda  short_debt  curr_debt   lt_debt  profit_lct   ocf_lct  cash_debt   fcf_ocf  lt_ppent   dltt_be  debt_assets  debt_capital  de_ratio    intcov  intcov_ratio  cash_ratio  quick_ratio  curr_ratio  cash_conversion  inv_turn   at_turn  rect_turn  pay_turn  sale_invcap  sale_equity  sale_nwc   rd_sale  adv_sale  staff_sale   accrual  ret_crsp    mktcap     price       ptb  peg_trailing  divyield    ireccd
datadate   cusip                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
1999-12-27 00036110        0.348065 -0.214940 -0.210398       -0.000285         0.032183  0.009029 -0.082164 -0.077413  0.010681     0.106125   0.095499  0.099825  0.086742 -0.239785 -0.970172  0.035710  0.065479  0.063974  0.065575  0.026644  0.065239  0.050039  0.065968  0.046769  0.003563 -0.025412   0.003486        0.183622       0.011375     0.024839        0.022314 -0.069911       0.010316     0.030631       -0.087080       0.085890 -0.054202    -0.020256 -0.173832  1.547744 -0.225775  0.032292    -0.019511   -0.839831  -0.570112  0.618209    0.212992 -0.012293   0.106701  0.031460 -0.193677 -0.083499    -0.015522      0.024029 -0.048341  0.002122     -0.017759   -0.239173    -0.202114   -0.010841        -0.045348 -0.153733  0.183417  -0.072447 -0.098343     0.083826    -0.011050 -0.015139 -0.068546 -0.060054   -0.300891  0.577670 -0.331502 -0.122514 -0.016268 -0.377466     -0.138488 -0.233073 -0.956710
           00095710       -0.136851 -0.401006  0.132732       -0.006469         0.030549  0.011240 -0.122822 -0.232862  0.011543     0.071412   0.058645  0.070429  0.057010 -0.243458  0.077903  0.145139  0.064652  0.061873  0.063815  0.025455  0.064377  0.049284  0.092598  0.055601  0.011663 -0.008133   0.006061        0.215172       0.013907     0.041087        0.035020  0.324305       0.299749    -0.276301       -0.248595      -0.332289 -0.055311    -0.024324 -0.177283 -0.744607  1.982465 -0.429756    -0.084237   -0.228287   0.165172 -0.716250    0.190059  0.142519   0.140681  0.042353 -0.115985 -0.138155    -0.030564     -0.186401 -0.056834  0.017521     -0.001348   -0.247897    -0.099676   -0.169760        -0.132342  0.093207  0.837812  -0.078378  0.572024     1.125812     0.022257 -0.015139 -0.068546 -0.060054   -0.300891  0.228412 -0.614869 -0.120588 -0.009417 -0.301178     -0.207469 -0.031416  0.928824
           00120410       -0.086330 -0.233944 -1.200586        0.016202         0.040853 -0.000101  0.104004  0.149672  0.009135     0.188031   0.178498  0.126793  0.114018 -0.222051 -0.028663  0.566178  0.067341  0.071578  0.069439  0.028061  0.068104  0.053609  0.053452  0.040134 -0.001855 -0.021548   0.000753        0.147131

## investigate correlation between factors and monthly returns for an initial filtering. we start with 79 factors and want to filter them down to about 5.

In [20]:
c = target_features.corr().loc[:,'monthly_return']
c= c[abs(c)>0.01].sort_values(ascending=False)
c

monthly_return     1.000000
rd_sale            0.028226
bm                 0.023205
at_turn            0.019865
sale_equity        0.019423
invt_act           0.015909
sale_invcap        0.014107
cash_conversion    0.013716
roa                0.011609
int_totdebt        0.010640
debt_ebitda       -0.010680
rsi               -0.010983
evm               -0.011395
ret_crsp          -0.011945
gpm               -0.012191
equity_invcap     -0.012647
pe_inc            -0.015103
peg_trailing      -0.016322
cfm               -0.017133
pe_op_dil         -0.017665
pe_op_basic       -0.018291
npm               -0.018571
accrual           -0.019035
ptpm              -0.019640
dpr               -0.020145
roe               -0.020439
opmad             -0.020459
opmbd             -0.021663
pe_exi            -0.021806
pcf               -0.022793
ptb               -0.024616
be                -0.029373
mktcap            -0.032231
ps                -0.038284
price             -0.039690
Name: monthly_return

## pick factors with stronger correlation to monthly returns

In [21]:
keep_col = ['monthly_return','rd_sale','bm','at_turn','sale_equity','invt_act','sale_invcap','cash_conversion','roa','int_totdebt','debt_ebitda','rsi','evm','ret_crsp','gpm','equity_invcap','pe_inc','peg_trailing','cfm','pe_op_dil','pe_op_basic','npm','accrual','ptpm','dpr','roe','opmad','opmbd','pe_exi','pcf','ptb','be','mktcap','ps','price']

## run individual regressions against monthly returns to investigate the relationship of the factor with monthly returns

In [22]:
import pandas as pd
import statsmodels.api as sm

# Assuming target_features is your DataFrame and keep_col is the list of variables
keep_col = ['monthly_return', 'rd_sale', 'bm', 'at_turn', 'sale_equity', 'invt_act', 
            'sale_invcap', 'cash_conversion', 'roa', 'int_totdebt', 'debt_ebitda', 'rsi', 
            'evm', 'ret_crsp', 'gpm', 'equity_invcap', 'pe_inc', 'peg_trailing', 'cfm', 
            'pe_op_dil', 'pe_op_basic', 'npm', 'accrual', 'ptpm', 'dpr', 'roe', 'opmad', 
            'opmbd', 'pe_exi', 'pcf', 'ptb', 'be', 'mktcap', 'ps', 'price']

# Remove the dependent variable from the list of independent variables
dependent_variable = 'monthly_return'
independent_vars = [col for col in keep_col if col != dependent_variable]

# Create a list to store the results
results = []

# Loop through each independent variable
for col in independent_vars:
    X = target_features[[col]]  # Independent variable
    y = target_features[dependent_variable]  # Dependent variable
    
    # Add a constant to the independent variable
    X = sm.add_constant(X)
    
    # Perform the regression
    model = sm.OLS(y, X).fit()
    
    # Extract the coefficient and t-statistic
    coef = model.params[col]  # Coefficient of the independent variable
    t_stat = model.tvalues[col]  # T-statistic for the independent variable
    p_value = model.pvalues[col]  # P-value for the independent variable
    r_squared = model.rsquared  # R-squared of the model
    adj_r_squared = model.rsquared_adj  # Adjusted R-squared of the model
    f_statistic = model.fvalue  # F-statistic of the model
    f_pvalue = model.f_pvalue  # P-value for the F-statistic
    std_err = model.bse[col]  # Standard error of the coefficient
    
    # Append results
    results.append({
        'Variable': col,
        'Coefficient': coef,
        'T-Statistic': t_stat,
        'P-Value': p_value,
        'Standard Error': std_err,
        'R-Squared': r_squared,
        'Adjusted R-Squared': adj_r_squared,
        'F-Statistic': f_statistic,
        'F-Statistic P-Value': f_pvalue
    })
# Convert the results list to a DataFrame
results_df = pd.DataFrame(results)

# Display the results
results_df.sort_values(by='Coefficient', ascending=False).loc[:,['Variable', 'Coefficient']]


,Variable,Coefficient
0,rd_sale,1.230275
6,cash_conversion,0.128425
8,int_totdebt,0.092962
3,sale_equity,0.056770
7,roa,0.044309
1,bm,0.034844
5,sale_invcap,0.029962
2,at_turn,0.021919
4,invt_act,0.014962
10,rsi,-0.008989


## pick the factors whose ecoefficient sign makes intuitive economic sense

In [ ]:
coefficients_that_make_sense = ['rd_sale', 'sale_equity', 'roa', 'bm', 'sale_invcap', 'at_turn', 'invt_act', 'accrual', 'ps'] 
#took out cfm because its correlated with rd spending
filtered_factors = target_features[coefficients_that_make_sense]

# Compute the correlation matrix
filtered_factors.corr()


,rd_sale,int_totdebt,sale_equity,roa,bm,sale_invcap,at_turn,invt_act,accrual,ps
rd_sale,1.000000,-0.000270,0.008892,0.233249,-0.091030,0.020482,0.027347,0.027519,0.112599,0.110767
int_totdebt,-0.000270,1.000000,0.004124,0.029773,0.032344,0.037386,0.006696,-0.008399,0.002664,-0.006647
sale_equity,0.008892,0.004124,1.000000,0.086315,-0.158743,0.441831,0.379142,0.120674,-0.015900,-0.139627
roa,0.233249,0.029773,0.086315,1.000000,-0.319227,0.122024,0.200470,-0.000674,0.081215,0.157206
bm,-0.091030,0.032344,-0.158743,-0.319227,1.000000,-0.151784,-0.153056,0.017682,-0.020415,-0.268037
sale_invcap,0.020482,0.037386,0.441831,0.122024,-0.151784,1.000000,0.769252,0.287030,0.007268,-0.211108
at_turn,0.027347,0.006696,0.379142,0.200470,-0.153056,0.769252,1.000000,0.427941,0.013314,-0.282535
invt_act,0.027519,-0.008399,0.120674,-0.000674,0.017682,0.287030,0.427941,1.000000,0.051595,-0.265193
accrual,0.112599,0.002664,-0.015900,0.081215,-0.020415,0.007268,0.013314,0.051595,1.000000,0.014195
ps,0.110767,-0.006647,-0.139627,0.157206,-0.268037,-0.211108,-0.282535,-0.265193,0.014195,1.000000


## run a multivariate regression with the factors that make intuitive economic sense and are not correlated to any other factor

In [28]:
import statsmodels.api as sm
coefficients_that_make_sense = ['rd_sale', 'sale_equity', 'roa', 'bm', 'sale_invcap', 'at_turn', 'invt_act', 'accrual', 'ps'] 
# Define the independent variables and the dependent variable
X = target_features[coefficients_that_make_sense]  # Independent variables
y = target_features['monthly_return']  # Dependent variable

# Add a constant to the independent variables
X = sm.add_constant(X)

# Fit the regression model
final_model = sm.OLS(y, X).fit()

# Display the regression summary
print(final_model.summary())


                            OLS Regression Results                            
Dep. Variable:         monthly_return   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     373.3
Date:                Sat, 30 Nov 2024   Prob (F-statistic):               0.00
Time:                        18:45:56   Log-Likelihood:            -1.0254e+06
No. Observations:              871346   AIC:                         2.051e+06
Df Residuals:                  871336   BIC:                         2.051e+06
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const          -0.0284      0.004     -7.845      

## choose the factors with the largest magnitude of coefficient

In [29]:
selected_factors = ['rd_sale', 'ps', 'bm', 'roa', 'accrual', 'sale_equity']